Using 1 late day for Homework 3
Dongshu Yang
4185086243
Flu1d1

1.(b) split training and test dataset

In [68]:
import os
import pandas as pd
import re
import numpy as np
from pathlib import Path
from scipy.stats import bootstrap

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

source_dir = "data/AReM" 
target_dir = "data/Splitted datasets"  
os.makedirs(os.path.join(target_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(target_dir, "test"), exist_ok=True)

for activity in ['bending1','bending2','cycling','lying','sitting','standing','walking']:
    activity_path = os.path.join(source_dir, activity)
    files = sorted([f for f in os.listdir(activity_path)], key=natural_sort_key)
    test_files = files[:3] if "bending" not in activity else files[:2]
    train_files = files[len(test_files):]
    for file in test_files:
        df = pd.read_csv(os.path.join(activity_path, file), skiprows=4, usecols=range(7)) 
        df.to_csv(os.path.join(target_dir, "test", f"{activity}_{file}"), index=False)
        
    for file in train_files:
        df = pd.read_csv(os.path.join(activity_path, file), skiprows=4, usecols=range(7))
        df.to_csv(os.path.join(target_dir, "train", f"{activity}_{file}"), index=False)
print('Train and test set are stored in folder splitted datasets')

Train and test set are stored in folder splitted datasets


1.(c) feature extraction i. ii.

In [69]:
datasets_dir = "data/Splitted datasets"  
feature_dir = "data/Feature extracted"
os.makedirs(feature_dir, exist_ok=True)

def extract_features(file_path):
    df = pd.read_csv(file_path, usecols=range(7))
    features = {}
    for col in df.columns:
        features[f"{col}_min"] = df[col].min()
        features[f"{col}_max"] = df[col].max()
        features[f"{col}_mean"] = df[col].mean()
        features[f"{col}_median"] = df[col].median()
        features[f"{col}_std"] = df[col].std()
        features[f"{col}_q1"] = df[col].quantile(0.25)
        features[f"{col}_q3"] = df[col].quantile(0.75)
    feature_df = pd.DataFrame([features])
    feature_df.insert(0, "instance", os.path.basename(file_path)) 
    return feature_df

for dataset in ["train", "test"]:
    feature_list = []
    dataset_path = os.path.join(datasets_dir, dataset)
    for file in os.listdir(dataset_path):
        if file == ".ipynb_checkpoints":   #ignore checkpoint folder
            continue
        file_path = os.path.join(dataset_path, file)
        if file == 'bending2_dataset4.csv':    #cleaning bending2_dataset4, which has a wrong form
            with open(file_path,"r",encoding = 'utf-8') as f:
                lines = f.readlines()
            title = lines[0].strip()
            datas = [line.strip().replace(" ", ",") for line in lines[1:]]
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(title + "\n") 
                f.write("\n".join(datas)) 
        feature_list.append(extract_features(file_path))
    all_features = pd.concat(feature_list, ignore_index=True)
    all_features.to_csv(os.path.join(feature_dir, f"{dataset}_features.csv"), index=False)
print('all features are stored in test_features.csv and train_features.csv under folder Features extracted')

all features are stored in test_features.csv and train_features.csv under folder Features extracted


1.(c) feature extraction iii. iv.

In [70]:
def compute_bootstrap_ci(df, confidence_level=0.90):
    ci_results = {}
    for col in df.columns[1:]:  
        data = df[col].values
        std_dev = np.std(data, ddof=1) 
        res = bootstrap((data,), np.std, confidence_level=confidence_level, n_resamples=1000, method='percentile')
        ci_results[col] = {
            "std": std_dev,
            "ci_lower": res.confidence_interval.low,
            "ci_upper": res.confidence_interval.high
        }
    return pd.DataFrame(ci_results).T

train_features = pd.read_csv(os.path.join(feature_dir, "train_features.csv"))
test_features = pd.read_csv(os.path.join(feature_dir, "test_features.csv"))
train_ci = compute_bootstrap_ci(train_features)
test_ci = compute_bootstrap_ci(test_features)
train_ci.to_csv(os.path.join(feature_dir, "train_std_ci.csv"))
test_ci.to_csv(os.path.join(feature_dir, "test_std_ci.csv"))

# choosing features(min, mean, max)
important_features = ["min", "mean", "max"]
selected_features = [col for col in train_features.columns if any(feature in col for feature in important_features)]
train_selected = train_features[["instance"] + selected_features]
test_selected = test_features[["instance"] + selected_features]
train_selected.to_csv(os.path.join(feature_dir, "train_selected_features.csv"), index=False)
test_selected.to_csv(os.path.join(feature_dir, "test_selected_features.csv"), index=False)

print("std and bootstrap ci stored in folder feature extracted")


std and bootstrap ci stored in folder feature extracted


ISLR 3.7.4
I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y=β0+β1X+β2X2+β2X3+ϵ.

(a)
Suppose that the true relationship between X and Y is linear, i.e. Y=β0+β1X+ϵ. Consider the training residual sum of squares (RSS)for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Because the true distribution is linear, then linear regression has a lower RSS.

(b)
Answer (a) using test rather than training RSS.

Linear regression is expected to perform better in both test and training data.

(c)
Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

In this case cubic regression will be better.

(d) Answer (c) using test rather than training RSS.

We'll need exact data to see which is better. Because maybe it's only a little far from linear in that case linear will be better.